In [ ]:
import numpy as np
import torch
import os
import logging
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
import seaborn as sns
import pandas as pd
import son
import json
import itertools
from ase.io import read
import shutil
from IPython.display import display
from PIL import Image

from nequip.data import AtomicDataDict
from nequip.nn import (
    GraphModuleMixin,
    SequentialGraphNetwork,
    AtomwiseLinear,
    AtomwiseReduce,
    ForceOutput,
    PerSpeciesScaleShift,
    ConvNetLayer,
)
from nequip.nn.embedding import (
    OneHotAtomEncoding,
    RadialBasisEdgeEncoding,
    SphericalHarmonicEdgeAttrs,
)

from nequip.utils import Config
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
from nequip.data import AtomicData

In [ ]:








class Phonons:
    
    def __init__(self,Path,PathToModel):
        
        self.ParentPath=os.getcwd()
        self.Path=os.path.join(self.ParentPath,Path)
        self.PathToModel=os.path.join(self.ParentPath,PathToModel)
        self.PathToPrediction=os.path.join(self.Path,"ML/")
        
        if os.path.exists(self.PathToPrediction):
            shutil.rmtree(self.PathToPrediction)
        os.mkdir(self.PathToPrediction); print(f"Created {self.PathToPrediction}")
        
    def GetNumberOfAtoms(self):
        os.chdir(self.Path)
        os.system("vibes u traj 2db trajectory.son")
        os.chdir("../")
        atoms=read(self.Path+'trajectory.db',index=':')
        self.NumberOfAtoms=len(atoms[0].numbers)
        print(f"Number of atoms = {len(atoms[0].numbers)}")
        return np.array(atoms[0].numbers),np.array(atoms[0].get_cell())
    
    def PredictPhonons(self):
        
        torch.cuda.empty_cache()


        config = Config.from_file(self.PathToModel+'tutorial.yaml')
        final_model=torch.load(self.PathToModel+'deployed.pth')
        final_model.eval()
        
        os.system('rm -rf '+self.PathToPrediction+'trajectoryML.son')
        metadata,data=son.load(self.Path+'trajectory.son')

        file1 = open(self.PathToPrediction+"trajectoryML.son","a")
        file1.write(json.dumps(metadata) + "\n" + "===" + "\n" )
        file1.close()
        file1 = open(self.PathToPrediction+"trajectoryML.son","a")
        
        z,cell=self.GetNumberOfAtoms()
        print(np.array(z))

        for i in data:

                r=np.array(i['atoms']['positions']).tolist()

                data = AtomicData.from_points(
                pos=r,
                r_max=config['r_max'],
                **{AtomicDataDict.ATOMIC_NUMBERS_KEY: torch.Tensor(torch.from_numpy(z.astype(np.float32)))\
                   .to(torch.int64)},pbc=True,cell=cell\
                )
                data = data.to('cpu')

                prediction = final_model(AtomicData.to_AtomicDataDict(data))['forces']
                prediction=prediction.cpu()
                
                idxsToZeroOut=np.abs(prediction) < 1e-4
                #print(idxsToZeroOut)
                prediction[idxsToZeroOut]=0.0
                #prediction[idxsToZeroOut]=0.0*prediction[idxsToZeroOut]
                
                forces=(prediction).tolist()
                i['calculator']['forces']=forces
                file1.write(json.dumps(i)+ "\n" + "---" + "\n" )
                
        file1.close()

        PredictedMetadata,PredictedData=son.load(self.PathToPrediction+'trajectoryML.son')
        metadata,data=son.load(self.Path+'trajectory.son')

        predicted_forces=[]
        actual_forces=[]
        for i in range(len(data)):
            predicted_forces.append(PredictedData[i]['calculator']['forces'])
            actual_forces.append(data[i]['calculator']['forces'])


        predicted_forces=list(itertools.chain(*predicted_forces))
        actual_forces=list(itertools.chain(*actual_forces))
        predicted_forces=list(itertools.chain(*predicted_forces))
        actual_forces=list(itertools.chain(*actual_forces))        

        plt.figure()
        plt.plot(predicted_forces,actual_forces,'o',label='R2='+str(r2_score(actual_forces,predicted_forces)))
        plt.xlabel('predicted F (eV/Ang)')
        plt.ylabel('actual F (eV/Ang)')
        plt.legend()    
        plt.savefig(self.PathToPrediction+"ActualVsPredictedForcesRegression.png")
        
        os.chdir(self.Path)
        os.system("vibes output phonopy trajectory.son --full")
        os.system("pdftoppm -png output/bandstructure.pdf > BandStructureVibes.png")
        
        self.PathToBandStructureVibes=self.Path+"BandStructureVibes.png"
        
        os.chdir(self.PathToPrediction)
        os.system("vibes output phonopy trajectoryML.son --full")
        os.system("pdftoppm -png output/bandstructure.pdf > BandStructurePrediction.png")
        self.PathToBandStructurePrediction=self.PathToPrediction+"BandStructurePrediction.png"
        os.chdir(self.ParentPath)
        
        
        self.PredictedForces=predicted_forces; self.ActualForces=actual_forces
        
    def BandStructureVibes(self):
        print("BS using Vibes")
        display(Image.open(self.PathToBandStructureVibes).resize((400,300)))

    def BandStructurePrediction(self):
        print("BS using NN")
        display(Image.open(self.PathToBandStructurePrediction).resize((400,300)))     


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


phonons=Phonons(Path="/home/villarreal/Documents/STO_MD/Phonons/phonopy2x2x2/",\
    PathToModel="/home/villarreal/Documents/STO_MD/ModelsAndData/nequip_training_RS_1500K_pbc_debug_Rchigher/")
#phonons2x1x1=Phonons(Path="phonopy3x3x3/",PathToModel="../nvt.1500.TRAINEDLONGER/")
#phonons2x1x1=Phonons(Path="phonopy3x2x2/",PathToModel="nequip_training_RS_3000K/")      
#phonons2x1x1=Phonons(Path="phonopy3x3x3/",PathToModel='nequip_training_RS_OCT10_ThisDataSetIsTheSameAsRs-Model1ButJustHasMoreRepetitionsAndYouWantedToSeeHowMoreRepetitionsPerformed/')
phonons.PredictPhonons()
phonons.BandStructureVibes()
phonons.BandStructurePrediction()